In [1]:
import os
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import scipy.cluster.hierarchy as spc
from sklearn.utils import resample

pio.templates.default = "plotly_white"

In [2]:
try:
    _ = first_run
except NameError:
    first_run = True
    os.chdir(os.getcwd().rsplit("/", 1)[0])
    from _aux import functions as func

# Load data

In [3]:
default = (
    pd.read_csv(
        "../data/train/X_train.csv",
        index_col=0,
        usecols=[
            "row_id",
            "num_arch_dc_0_12m",
            "num_arch_dc_12_24m",
            "num_arch_ok_0_12m",
            "num_arch_ok_12_24m",
            "num_arch_rem_0_12m",
            "num_arch_written_off_0_12m",
            "num_arch_written_off_12_24m",
        ],
    )
    .join(pd.read_csv("../data/train/y_train.csv", index_col=0))
    .query("default == 1")
)

not_default = (
    pd.read_csv(
        "../data/train/X_train.csv",
        index_col=0,
        usecols=[
            "row_id",
            "num_arch_dc_0_12m",
            "num_arch_dc_12_24m",
            "num_arch_ok_0_12m",
            "num_arch_ok_12_24m",
            "num_arch_rem_0_12m",
            "num_arch_written_off_0_12m",
            "num_arch_written_off_12_24m",
        ],
    )
    .join(pd.read_csv("../data/train/y_train.csv", index_col=0))
    .query("default == 0")
)

df = pd.read_csv(
    "../data/train/X_train.csv",
    index_col=0,
).join(pd.read_csv("../data/train/y_train.csv", index_col=0))

## The problematic variable

Inspite of it seeming to be a good idea to keep track of invoices that have not being paid and become loss, both variables that strive for this function are utterly problematic. Both "num_arch_written_off_0_12m" and "num_arch_written_off_12_24m" have 18% of missing values, which in itself is not a problem. The issue here lies in the fact that less than 0.1% of observations are non-zero, which means that it has an extremely low signal-to-noise ratio.


In [4]:
df[["num_arch_written_off_0_12m", "num_arch_written_off_12_24m"]].replace(
    0, np.nan
).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71980 entries, 27330 to 16866
Data columns (total 2 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   num_arch_written_off_0_12m   5 non-null      float64
 1   num_arch_written_off_12_24m  8 non-null      float64
dtypes: float64(2)
memory usage: 3.7 MB


But perhaps, those cases in which they are non-zero must be an extremely good predictor of default, right? Wrong!

In [5]:
pd.concat(
    [
        (
            df.query("num_arch_written_off_0_12m > 0")
            .agg(
                default=("default", "sum"),
                not_default=("default", func.complement),
                count=("default", "count"),
            )
            .squeeze()
            .to_frame(name="num_arch_written_off_0_12m")
        ),
        (
            df.query("num_arch_written_off_12_24m > 0")
            .agg(
                default=("default", "sum"),
                not_default=("default", func.complement),
                count=("default", "count"),
            )
            .squeeze()
            .to_frame(name="num_arch_written_off_12_24m")
        ),
    ],
    axis=1,
).T

,default,not_default,count
num_arch_written_off_0_12m,1.0,4.0,5.0
num_arch_written_off_12_24m,0.0,8.0,8.0


Therefore, we will drop these variables from our exploration

In [6]:
default = default.drop(
    ["num_arch_written_off_0_12m", "num_arch_written_off_12_24m"], axis=1
)
not_default = not_default.drop(
    ["num_arch_written_off_0_12m", "num_arch_written_off_12_24m"], axis=1
)
df = df.drop(["num_arch_written_off_0_12m", "num_arch_written_off_12_24m"], axis=1)

## Correlation between "archived" variables

As shown in our sanity profile report, "archieved" variables have high correlation amongst themselves mainly due to overlaping lookback windows for aggregation. The first thing we must do is choose one (or some) of them to represent the group, which can be achieved by correlation clustering.

In [7]:
corr = pd.concat([default, not_default]).corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        z=corr.mask(mask),
        x=corr.columns,
        y=corr.columns,
        colorscale=px.colors.diverging.RdBu,
        zmin=-1,
        zmax=1,
    )
)

fig.update_layout(
    title="Correlation between 'account' variables",
    yaxis_autorange="reversed",
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    width=1000,
    height=500,
)

# fig.update_traces(opacity=0.6)
fig.show()

In [8]:
pdist = spc.distance.pdist(corr)
linkage = spc.linkage(pdist, method="single")
idx = spc.fcluster(linkage, 0.6 * pdist.max(), "distance")

columns = [default.columns.tolist()[i] for i in list((np.argsort(idx)))]
clusterd_corr = pd.concat([default, not_default]).reindex(columns, axis=1).corr()

mask = np.zeros_like(clusterd_corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

corr["default"].to_frame(name="corr_with_label").assign(cluster=idx).drop(
    "default"
).sort_values(["cluster", "corr_with_label"], ascending=[True, False])

,corr_with_label,cluster
num_arch_ok_12_24m,-0.043306,1
num_arch_ok_0_12m,-0.046540,1
num_arch_dc_0_12m,0.113312,2
num_arch_dc_12_24m,0.104962,2
num_arch_rem_0_12m,0.001826,2


Let's get an overview on our clusters:

In [9]:
pd.concat(
    [
        df[["num_arch_ok_0_12m", "num_arch_ok_12_24m"]],
        df[["num_arch_dc_0_12m", "num_arch_dc_12_24m", "num_arch_rem_0_12m"]],
    ],
    keys=["cluster_1", "cluster_2"],
    axis=1,
).describe(np.append(np.arange(0.25, 1.0, 0.1), np.array([0.99])))

cluster_1                            cluster_2  \
      num_arch_ok_0_12m num_arch_ok_12_24m num_arch_dc_0_12m   
count      71980.000000       71980.000000      71980.000000   
mean           7.306335           6.373979          0.061406   
std           15.936948          15.265276          0.374491   
min            0.000000           0.000000          0.000000   
25%            0.000000           0.000000          0.000000   
35%            1.000000           0.000000          0.000000   
45.0%          2.000000           1.000000          0.000000   
50%            2.000000           2.000000          0.000000   
55.0%          3.000000           2.000000          0.000000   
65.0%          5.000000           4.000000          0.000000   
75.0%          7.000000           6.000000          0.000000   
85.0%         12.000000          11.000000          0.000000   
95.0%         31.000000          28.000000          0.000000   
99%           81.000000          73.000000          2.000000   
max          248.000000         313.000000         16.000000   

                                             
      num_arch_dc_12_24m num_arch_rem_0_12m  
count       71980.000000       71980.000000  
mean            0.059906           0.466143  
std             0.371457           1.337338  
min             0.000000           0.000000  
25%             0.000000           0.000000  
35%             0.000000           0.000000  
45.0%           0.000000           0.000000  
50%             0.000000           0.000000  
55.0%           0.000000           0.000000  
65.0%           0.000000           0.000000  
75.0%           0.000000           0.000000  
85.0%           0.000000           1.000000  
95.0%           0.000000           2.000000  
99%             2.000000           5.000000  
max            13.000000          42.000000

## 1. "num_arch_dc_0_12m"

Despite having higher correlation with the target label, "cluster 2" variables have mostly zero values. Let's check how defaults behave across values.

In [10]:
func.test_k_prop(
    df.groupby("num_arch_dc_0_12m").agg(
        default=("default", "sum"),
        not_default=("default", func.complement),
        count=("default", "count"),
        contamination=("default", lambda s: s.sum() / s.shape[0]),
    )
)[1]

Using 13 degrees of freedom
Reject null hypothesis with 1438.8705686391422 > 22.362032494826945


,default,not_default,count,contamination,expected_default,expected_not_default,chi_default,chi_not_default
num_arch_dc_0_12m,,,,,,,,
0,808.0,68126.0,68934,0.011721,986.413170,67947.586830,32.269702,0.468468
1,127.0,2165.0,2292,0.055410,32.797444,2259.202556,270.573575,3.927988
2,56.0,427.0,483,0.115942,6.911503,476.088497,348.647820,5.061413
3,24.0,117.0,141,0.170213,2.017644,138.982356,239.499156,3.476873
4,3.0,50.0,53,0.056604,0.758405,52.241595,6.625414,0.096183
5,1.0,21.0,22,0.045455,0.314810,21.685190,1.491332,0.021650
6,1.0,27.0,28,0.035714,0.400667,27.599333,0.896506,0.013015
7,2.0,9.0,11,0.181818,0.157405,10.842595,21.569585,0.313131
8,2.0,1.0,3,0.666667,0.042929,2.957071,89.220922,1.295244


We can observe that contamination rates increase, with some exceptions, for higher number of archived invoices of status "dc". Could we gain any benefit from transforming it into a binned variable?

In [11]:
(
    df.assign(var_bool=pd.cut(df["num_arch_dc_0_12m"], [-1, 1, 5, np.inf]))
    .groupby("var_bool")
    .agg(
        default=("default", "sum"),
        not_default=("default", func.complement),
        count=("default", "count"),
        contamination=("default", lambda s: s.sum() / s.shape[0]),
    )
)

,default,not_default,count,contamination
var_bool,,,,
"(-1.0, 1.0]",935.0,70291.0,71226,0.013127
"(1.0, 5.0]",84.0,615.0,699,0.120172
"(5.0, inf]",11.0,44.0,55,0.200000


Once again we see that binning the variable does provide subclasses with much higher contamination rate.
We conclude that, despite having a high concentrarion of zero values, this variable (and its binned counterpart) could be useful as features for our models.

## 2. "num_arch_ok_0_12m"

This variable has too many unique values to be considered categorical. We can take 3 different approaches to explore it: numerical, boolean and binned.

Let's first look at it as a numerical variable and verify whether its behavious for default and not_default differs significantly.


In [12]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=not_default.num_arch_ok_0_12m.sample(1000, replace=True, random_state=42),
        name="not_default",
        histfunc="count",
        # histnorm='probability',
        xbins=dict(start=-1, end=50, size=1),
    )
)

fig.add_trace(
    go.Histogram(
        x=default.num_arch_ok_0_12m.sample(1000, replace=True, random_state=42),
        name="default",
        histfunc="count",
        # histnorm='probability',
        xbins=dict(start=-1, end=50, size=1),
    )
)

fig.update_layout(title="Title", barmode="overlay")

fig.update_traces(opacity=0.6)
fig.show()

Apparently, customers who default tend to have smaller values for "num_arch_ok_0_12m", which is in line with common sense. Users who have made successfull purchases within the last 12 months should be expected to be more reliable payers. Of course, one could lose their job, fall ill and be a victim of financial crime, all of which would impair their ability to pay their debt. Thus, this variable is not expected to perfectly explain the target label, although it is a good candidate for feature.

Let's examine whether the difference between "default" and "not_default" is indeed significant.

In [28]:
num_iterations = 100_000
sample1 = []
sample2 = []
combined = np.concatenate(
    (default.num_arch_ok_0_12m, not_default.num_arch_ok_0_12m), axis=0
)

for i in range(num_iterations):
    np.random.seed(i)
    combined = np.concatenate(
        (
            default.num_arch_ok_0_12m.sample(1_000, replace=True),
            not_default.num_arch_ok_0_12m.sample(1_000, replace=True),
        ),
        axis=0,
    )
    sample1.append(resample(combined, n_samples=500))
    sample2.append(resample(combined, n_samples=500))

diff_bootstrap_means = np.mean(sample1, axis=1) - np.mean(sample2, axis=1)

observed_difference = np.mean(
    default.num_arch_ok_0_12m.sample(1_000, replace=True, random_state=42)
) - np.mean(
    not_default.num_arch_ok_0_12m.sample(1_000, replace=True, random_state=42)
)

p_value = (
    diff_bootstrap_means[diff_bootstrap_means < observed_difference].shape[0]
    / num_iterations
)

ci_lower, ci_upper = np.percentile(diff_bootstrap_means, [0.5, 99.5])

In [29]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=diff_bootstrap_means,
        name="sample_difference",
        histfunc="count",
        # xbins=dict(size=.05),
    )
)

fig.add_vline(
    x=observed_difference,
    line_width=3,
    line_color="indianred",
    line_dash="dash",
    annotation_text=f"Observed",
)

fig.add_vline(
    x=ci_lower,
    line_width=3,
    line_color="indianred",
    line_dash="solid",
    annotation_text=f"0.5%",
)

fig.add_vline(
    x=ci_upper,
    line_width=3,
    line_color="indianred",
    line_dash="solid",
    annotation_text=f"99.5%",
)

fig.update_layout(
    title="Distribution of difference in medians",
    barmode="overlay",
)

fig.update_traces(opacity=0.75)
# fig.update_xaxes(range=[-5, 5])
fig.show()

This result shows us that there is a significative difference in means, which in turn suggests that this variable could be useful for model induction. Now, we analyze whether its binned counterpart is too a good candidate.

In [16]:
(
    df[["default"]]
    .assign(
        bins_var=pd.cut(df["num_arch_ok_0_12m"], 10),
    )
    .groupby("bins_var")
    .agg(
        default=("default", "sum"),
        not_default=("default", func.complement),
        count=("default", "count"),
        contamination=("default", lambda s: s.sum() / s.shape[0]),
    )
)

,default,not_default,count,contamination
bins_var,,,,
"(-0.248, 24.8]",1021.0,66129.0,67150,0.015205
"(24.8, 49.6]",9.0,2999.0,3008,0.002992
"(49.6, 74.4]",0.0,964.0,964,0.000000
"(74.4, 99.2]",0.0,365.0,365,0.000000
"(99.2, 124.0]",0.0,227.0,227,0.000000
"(124.0, 148.8]",0.0,146.0,146,0.000000
"(148.8, 173.6]",0.0,75.0,75,0.000000
"(173.6, 198.4]",0.0,11.0,11,0.000000
"(198.4, 223.2]",0.0,26.0,26,0.000000


The fact that using 10 bins has very little effect on the contamination rate shows that this is not a good strategy to move forward with. Hopefully, its boolean counterpart can perform better.

In [17]:
(
    df[["default"]]
    .assign(
        bool_var=df["num_arch_ok_0_12m"] > 1,
    )
    .groupby("bool_var")
    .agg(
        default=("default", "sum"),
        not_default=("default", func.complement),
        count=("default", "count"),
        contamination=("default", lambda s: s.sum() / s.shape[0]),
    )
)

,default,not_default,count,contamination
bool_var,,,,
False,830.0,28880.0,29710,0.027937
True,200.0,42070.0,42270,0.004731


The boolean variable does increase contamination in 2 fold for the group "up to 1" at the expense of a very diluted rate for the "above 1" group, which wouldn't be a problem if the latter held few "default" observations. However, the group holds almost 20% of "default" observations, thus making the boolean variable also unsuitable.

--

To sum up, we show that "num_arch_dc_0_12m" and its boolean counterpart are good candidates for features.
Furthermore, we test three variations of "num_arch_ok_0_12m", which showed to be best in its original form.
Hence, we move forward with 3 candidates for features from "archieved" variables:
- num_arch_dc_0_12m
- num_arch_dc_0_12m_binned
- num_arch_ok_0_12m

Next, we look at "order" variables.